# 1. Setup and Trained Model Loading

This notebook is your interactive lab for analyzing the results of a trained model. First, we'll import the necessary libraries and load our best-performing model (`best.pt`) from a training run.

In [ ]:
# Import necessary libraries
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from pathlib import Path
from IPython.display import display

# --- Configuration ---
# ATTENTION: Change this path to your best trained model (best.pt)
MODEL_PATH = '../runs/train/yolov11n_seg_25_epochs/weights/best.pt' # Example path

# ATTENTION: Change this to the path of your original images
IMAGES_DIR = '../data/raw_data/train'

# Load the model
try:
    model = YOLO(MODEL_PATH)
    print(f"Model loaded successfully from: {MODEL_PATH}")
except Exception as e:
    print(f"ERROR loading model: {e}")

# 2. Run Prediction and Visualize Results

Let's use our loaded model to run a prediction on a random test image and see how well it performs visually.

In [ ]:
# Choose a random image to test
if 'model' in locals():
    test_image_path = random.choice(list(Path(IMAGES_DIR).glob("*.png")))
    print(f"Running prediction on image: {test_image_path.name}")

    # Run prediction
    results = model.predict(source=str(test_image_path), conf=0.25)
    result = results[0] # Get result for the first image

    # Display the image with predicted masks
    predicted_plot = result.plot(line_width=1, font_size=8)
    plt.figure(figsize=(12, 12))
    plt.imshow(cv2.cvtColor(predicted_plot, cv2.COLOR_BGR2RGB))
    plt.title(f"Model Predictions on '{test_image_path.name}'")
    plt.axis('off')
    plt.show()
else:
    print("Model not loaded. Please run the previous cell first.")

# 3. Morphometric Feature Extraction and Analysis

This is the core scientific analysis. We define a function to calculate key metrics from the predicted masks and then display the results in a table and a histogram.

In [ ]:
def calculate_morphometrics(result_object):
    """
    Calculates morphometric features from a YOLO result object.
    
    Args:
        result_object: The result object from a `model.predict()` call.
        
    Returns:
        pd.DataFrame: A pandas DataFrame with morphometric data for each cell.
    """
    masks = result_object.masks
    if not masks:
        return pd.DataFrame()

    morph_data = []
    class_names = result_object.names

    for i, mask_data in enumerate(masks):
        mask_np = mask_data.data[0].cpu().numpy().astype(np.uint8)
        contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours: continue
        
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        circularity = (4 * np.pi * area) / (perimeter**2) if perimeter > 0 else 0
        
        aspect_ratio = 0
        if len(cnt) >= 5:
            (x, y), (MA, ma), angle = cv2.fitEllipse(cnt)
            aspect_ratio = MA / ma if ma > 0 else 0
        
        class_id = int(result_object.boxes[i].cls)
        confidence = float(result_object.boxes[i].conf)

        morph_data.append({
            'cell_id': i + 1, 'class_name': class_names[class_id],
            'confidence': round(confidence, 4), 'area_pixels': area,
            'perimeter_pixels': perimeter, 'circularity': round(circularity, 4),
            'aspect_ratio': round(aspect_ratio, 4)
        })
    return pd.DataFrame(morph_data)

# Run the analysis on the prediction we just made
if 'result' in locals() and result.masks:
    df_morph = calculate_morphometrics(result)
    print("Morphometric Analysis Table:")
    display(df_morph)

    # Plot a histogram of the cell areas
    plt.figure(figsize=(10, 6))
    df_morph['area_pixels'].hist(bins=20, color='skyblue', edgecolor='black')
    plt.title('Distribution of Detected Cell Areas')
    plt.xlabel('Area (pixels²)')
    plt.ylabel('Frequency')
    plt.grid(axis='y', alpha=0.75)
    plt.show()
else:
    print("\nNo masks were detected in the test image to perform analysis.")